# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import math

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline as skPipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SequentialFeatureSelector


from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline

# Read data into DFs

In [2]:
# 1. Column names (15 features + label)
headers = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'pred'   # label (<=50K / >50K)
]

# 2. Read raw data
original_train_df = pd.read_csv(
    "census-income.data.csv",
    header=None,
    names=headers,
    index_col=False
)

original_test_df = pd.read_csv(
    "census-income.test.csv",
    header=None,
    names=headers,
    index_col=False
)

# Work on copies
train_df = original_train_df.copy()
test_df = original_test_df.copy()

In [3]:
# Transform target into binary

target_map = {
'<=50K': 0,
'>50K': 1,
'<=50K.': 0,
'>50K.': 1
}

train_df['pred'] = train_df['pred'].str.strip().map(target_map)
test_df['pred'] = test_df['pred'].str.strip().map(target_map)

In [4]:
# ---------- Helper Functions ----------

def data_to_str(df):
    """Strip leading/trailing spaces from all object columns."""
    df = df.copy()
    str_cols = df.select_dtypes(include=["object"]).columns
    for col in str_cols:
        df[col] = df[col].str.strip()
    return df

def replace_qmarks(df):
    """Convert '?' to np.nan."""
    return df.replace("?", np.nan)


def category_remapping(df):
    '''Maps categorical variables into more functional bins'''
    copy_df = df.copy()
    
    # workclass
    workclass_mapping = {
        'State-gov':'Government',
        'Local-gov':'Government',
        'Federal-gov':'Government',
        'Self-emp-inc':'Incorporated-Entrepreneur',
        'Self-emp-not-inc':'Unincorporated-Entrepreneur',
        'Without-pay':'Unemployed',
        'Never-worked':'Unemployed',
        'Private':'Private'
    }
    if "workclass" in copy_df:
        copy_df["workclass-cat"] = copy_df["workclass"].map(workclass_mapping)
    
    # education
    edu_mapping = {
        'Preschool':'HS-dropout',
        '1st-4th':'HS-dropout',
        '5th-6th':'HS-dropout',
        '7th-8th':'HS-dropout',
        '9th':'HS-dropout',
        '10th':'HS-dropout',
        '11th':'HS-dropout',
        '12th':'HS-dropout',
        'HS-grad':'HS-grad',
        'Some-college':'Some-college',
        'Assoc-acdm':'Some-college',
        'Assoc-voc':'Some-college',
        'Bachelors':'Bachelors',
        'Masters':'Advanced-degree',
        'Prof-school':'Advanced-degree',
        'Doctorate':'Advanced-degree'
    }
    if "education" in copy_df:
        copy_df["education-cat"] = copy_df["education"].map(edu_mapping)
    
    # marital-status
    marital_mapping = {
        'Never-married':'Single/Unmarried',
        'Divorced':'Single/Unmarried',
        'Separated':'Single/Unmarried',
        'Widowed':'Single/Unmarried',
        'Married-spouse-absent':'Single/Unmarried',
        'Married-civ-spouse':'Married',
        'Married-AF-spouse':'Married'
    }
    if "marital-status" in copy_df:
        copy_df["marital-cat"] = copy_df["marital-status"].map(marital_mapping)
    
    # occupation
    occupation_mapping = {
        'Exec-managerial':'White-collar',
        'Prof-specialty':'White-collar',
        'Tech-support':'White-collar',
        'Other-service':'Service',
        'Sales':'Service',
        'Adm-clerical':'Service',
        'Protective-serv':'Service',
        'Craft-repair':'Blue-collar',
        'Transport-moving':'Blue-collar',
        'Machine-op-inspct':'Blue-collar',
        'Armed-Forces':'Military',
        'Priv-house-serv':'Manual',
        'Farming-fishing':'Manual',
        'Handlers-cleaners':'Manual'
    }
    if "occupation" in copy_df:
        copy_df["occupation-cat"] = copy_df["occupation"].map(occupation_mapping)
    
    # native-country → native_imm_cat
    if "native-country" in copy_df:
        s = copy_df["native-country"]
        native_imm_cat = (
            s.map({"United-States": "Native"})  # US → Native, others NaN
             .fillna("Immigrant")              # non-US, non-missing → Immigrant
             .where(s.notna(), pd.NA)          # where original was missing, keep NA
        )
        copy_df["native_imm_cat"] = native_imm_cat.replace({pd.NA: np.nan})
    
    # hours-per-week binning
    if "hours-per-week" in copy_df:
        hrs_bins = [0, 30, 40, 60, 100]
        hrs_labels = ["Part-Time", "Underworked", "Full-Time+", "Overworked"]
        copy_df["hours_bin"] = pd.cut(copy_df["hours-per-week"], bins=hrs_bins, labels=hrs_labels)
    
    # capital-flow binning
    if "capital-gain" in copy_df and "capital-loss" in copy_df:
        copy_df["net-capital-flow"] = copy_df["capital-gain"] - copy_df["capital-loss"]
        cap_bins = [-10000, 10000, 99999]
        cap_labels = ["Standard", "High Net-Worth"]
        copy_df["cap_flow_bin"] = pd.cut(copy_df["net-capital-flow"], bins=cap_bins, labels=cap_labels)
    
    # Drop original columns we replaced
    drop_cols = [
        "workclass", "fnlwgt", "education", "education-num",
        "marital-status", "occupation", "native-country",
        "hours-per-week", "capital-gain", "capital-loss",
        "net-capital-flow"
    ]
    existing = [c for c in drop_cols if c in copy_df.columns]
    copy_df = copy_df.drop(columns=existing)
    
    return copy_df


class BasicPrep(BaseEstimator, TransformerMixin):
    """
    1) strip whitespace
    2) convert '?' → np.nan
    3) apply all category/bucket remappings & drop originals
    """
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = X.copy()
        df = data_to_str(df)
        df = replace_qmarks(df)
        df = category_remapping(df)
        return df


In [5]:
train_df_drop = train_df.copy()

train_df_drop = data_to_str(train_df_drop)
train_df_drop = replace_qmarks(train_df_drop)

### Train Test Split

In [6]:
target_col = "pred"

X_train_raw = train_df.drop(columns=[target_col])
y_train = train_df[target_col]

X_test_raw  = test_df.drop(columns=[target_col])
y_test  = test_df[target_col]

# Logistic Regression
## Build transformers, identify features

In [12]:
numeric_features = ["age"]

categorical_features = [
    "relationship", "race", "sex",
    "workclass-cat", "education-cat", "marital-cat",
    "occupation-cat", "native_imm_cat",
    "hours_bin", "cap_flow_bin"
]

# Numeric pipeline
numeric_transformer = skPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("log", FunctionTransformer(np.log1p, feature_names_out="one-to-one")),
    ("scaler", MinMaxScaler())
])

# Categorical pipeline
categorical_transformer = skPipeline(steps=[
    ("imputer", SimpleImputer(
        strategy="most_frequent",   # or "constant" with fill_value="Unknown"
        fill_value="Unknown"
    )),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="drop"
)

## Build pipeline & test using GridSearchCV

In [13]:
logreg_base = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    solver="lbfgs",
    penalty="l2"
)

logreg_pipeline = ImbPipeline(steps=[
    ("prep", BasicPrep()),
    ("preprocessor", preprocessor),
    ("sampler", "passthrough"),  # will be set to ROS or passthrough in grid
    ("clf", logreg_base),
])

logreg_param_grid = {
    # --- categorical missing-data ---
    "preprocessor__cat__imputer__strategy": ["most_frequent", "constant"],
    # "constant" uses fill_value="Unknown" already set above

    # --- numeric normalization on age ---
    "preprocessor__num__log": [
        "passthrough",
        FunctionTransformer(np.log1p, feature_names_out="one-to-one"),
    ],
    "preprocessor__num__scaler": [
        "passthrough",
        MinMaxScaler(),
    ],

    # --- imbalance handling ---
    "sampler": [
        "passthrough",
        RandomOverSampler(random_state=42),
    ],

    # --- LogisticRegression hyperparams ---
    "clf__C": [0.1, 1.0, 10.0],
}

logreg_grid = GridSearchCV(
    estimator=logreg_pipeline,
    param_grid=logreg_param_grid,
    cv=5,
    scoring="accuracy",  # or 'f1_macro'
    n_jobs=-1,
    verbose=1
)

logreg_grid.fit(X_train_raw, y_train)

print("Best params (LogReg):", logreg_grid.best_params_)
print("Best CV accuracy (LogReg):", logreg_grid.best_score_)

print("Test accuracy (LogReg):", logreg_grid.score(X_test_raw, y_test))
print(classification_report(y_test, logreg_grid.predict(X_test_raw)))


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best params (LogReg): {'clf__C': 10.0, 'preprocessor__cat__imputer__strategy': 'constant', 'preprocessor__num__log': FunctionTransformer(feature_names_out='one-to-one', func=<ufunc 'log1p'>), 'preprocessor__num__scaler': MinMaxScaler(), 'sampler': 'passthrough'}
Best CV accuracy (LogReg): 0.8424496310723857
Test accuracy (LogReg): 0.8436828204655734
              precision    recall  f1-score   support

           0       0.87      0.93      0.90     12435
           1       0.71      0.57      0.63      3846

    accuracy                           0.84     16281
   macro avg       0.79      0.75      0.77     16281
weighted avg       0.84      0.84      0.84     16281



# RandomForestClassifier
## Build transformers

In [9]:
numeric_transformer_rf = skPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    # no log, no scaler – RF is tree-based and scale-invariant
])

In [10]:
preprocessor_rf = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_rf, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="drop"
)


## Build pipeline & test using GridSearchCV

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf_base = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)

rf_pipeline = ImbPipeline(steps=[
    ("prep", BasicPrep()),
    ("preprocessor", preprocessor_rf),
    ("sampler", "passthrough"),
    ("clf", rf_base),
])

rf_param_grid = {
    # cat missing data
    "preprocessor__cat__imputer__strategy": ["most_frequent", "constant"],

    # imbalance handling
    "sampler": [
        "passthrough",
        RandomOverSampler(random_state=42),
    ],

    # RF hyperparams – trimmed
    "clf__n_estimators": [100, 200],       # 400 is overkill for a grid
    "clf__max_depth": [None, 20],          # drop the 10
    "clf__min_samples_leaf": [1, 2],       # drop the 5
}

rf_grid = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=rf_param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2
)

rf_grid.fit(X_train_raw, y_train)

print("Best params (RF):", rf_grid.best_params_)
print("Best CV accuracy (RF):", rf_grid.best_score_)

print("Test accuracy (RF):", rf_grid.score(X_test_raw, y_test))
print(classification_report(y_test, rf_grid.predict(X_test_raw)))

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__n_estimators=100, preprocessor__cat__imputer__strategy=most_frequent, sampler=passthrough; total time=   6.1s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__n_estimators=100, preprocessor__cat__imputer__strategy=constant, sampler=passthrough; total time=   6.0s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__n_estimators=200, preprocessor__cat__imputer__strategy=most_frequent, sampler=RandomOverSampler(random_state=42); total time=  18.8s


/Users/johnchester/Desktop/homework/env/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__n_estimators=100, preprocessor__cat__imputer__strategy=most_frequent, sampler=passthrough; total time=   6.4s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__n_estimators=100, preprocessor__cat__imputer__strategy=constant, sampler=RandomOverSampler(random_state=42); total time=   9.6s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__n_estimators=200, preprocessor__cat__imputer__strategy=most_frequent, sampler=RandomOverSampler(random_state=42); total time=  18.0s
[CV] END clf__max_depth=None, clf__min_samples_leaf=2, clf__n_estimators=100, preprocessor__cat__imputer__strategy=constant, sampler=passthrough; total time=   4.0s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__n_estimators=100, preprocessor__cat__imputer__strategy=most_frequent, sampler=RandomOverSampler(random_state=42); total time=   9.3s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__n_estimators=200, preprocessor_